In [1]:
import matplotlib.pyplot as plt
import os

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer

import tensorflow as tf
import pickle
import numpy as np

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import functions to create template datasets

In [4]:
from annsa.load_simulate_templates import (load_templates,
                                           create_template_parameters,
                                           simulate_template_dataset)


In [14]:
from annsa.model_classes import (dnn_model_features,
                                 DNN,
                                 save_model,
                                 train_earlystop)

from annsa import isotopes

#### Create template datasets

In [6]:
spectral_template_settings = ['no-shielding']

spectral_templates = load_templates(
    spectral_template_settings,
    templates_folder='../dataset_generation/templates/',
    spectrum_identifier = "_10uC_spectrum.spe"
    )

isotope_list = isotopes[:-3]

In [7]:
tmp = 10

template_parameters = create_template_parameters(integration_time_range=[60,3600],
                                                 integration_time_division=tmp,
                                                 signal_to_background_range=[0.5,3],
                                                 signal_to_background_division=tmp,
                                                 calibration_range=[0.8,1.2],
                                                 calibration_division=tmp,
                                                 print_divisions=True,
                                                 division_offset=False,
                                                 )

training_data, training_keys=simulate_template_dataset(isotope_list,
                                                       spectral_template_settings,
                                                       spectral_templates,
                                                       template_parameters)

integration_times
[  60.           94.56351601  149.03764267  234.89205847  370.20364886
  583.46264459  919.57131887 1449.29828556 2284.17902714 3600.        ]
signal_to_backgrounds
[0.5        0.61014247 0.74454766 0.9085603  1.10870244 1.35293289
 1.65096362 2.01464604 2.45844221 3.        ]
calibrations
[0.952      1.00488889 1.05777778 1.11066667 1.16355556 1.21644444
 1.26933333 1.32222222 1.37511111 1.428     ]
 Isotope U235, template no-shielding,0 total spectra simulated b[2kkkb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2

In [8]:
template_parameters = create_template_parameters(integration_time_range=[60,3600],
                                                 integration_time_division=tmp,
                                                 signal_to_background_range=[0.5,3],
                                                 signal_to_background_division=tmp,
                                                 calibration_range=[0.8,1.2],
                                                 calibration_division=tmp,
                                                 print_divisions=True,
                                                 division_offset=True,
                                                 )

testing_data, testing_keys=simulate_template_dataset(isotope_list,
                                                     spectral_template_settings,
                                                     spectral_templates,
                                                     template_parameters)

integration_times
[  77.281758    121.80057934  191.96485057  302.54785366  476.83314672
  751.51698173 1184.43480222 1866.73865635 2942.08951357]
signal_to_backgrounds
[0.55507123 0.67734507 0.82655398 1.00863137 1.23081767 1.50194826
 1.83280483 2.23654413 2.72922111]
calibrations
[0.97844444 1.03133333 1.08422222 1.13711111 1.19       1.24288889
 1.29577778 1.34866667 1.40155556]
 Isotope U235, template no-shielding,0 total spectra simulated b[2kkkb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kIsotope Co57, template no-shielding,0 total spectra simulated b[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kb[2kIsotope In111, template no-shielding,0 tota

#### Define scaling pipeline, binarize training keys

In [9]:
scaler=make_pipeline(FunctionTransformer(np.log1p, validate=True))
mlb=LabelBinarizer()
training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_keys.shape[0],1]))
testing_keys_binarized = mlb.transform(testing_keys.reshape([testing_keys.shape[0],1]))

### Train and save model

#### Load best network features from hyperparmater search

In [11]:
errors = np.load('../hyperparameter_search/hyperparameter-search-results/final_test_errors_vanilla_dnn.npy')
best_network_arg = np.argmin(errors)

model_id='vanilla_dnn_'+str(best_network_arg)+'_features'
with open('../hyperparameter_search/hyperparameter-search-results/'+model_id,"rb" ) as f:
        model_features=pickle.load(f)


In [13]:
num_epochs=20
model_features.scaler.fit(training_data)
train_dataset = (training_data, training_keys_binarized)
test_dataset = (testing_data, testing_keys_binarized)

num_networks_to_train = 5


for network_id in range(num_networks_to_train):

    tf.reset_default_graph()
    optimizer = tf.train.AdamOptimizer(model_features.learining_rate)
    model = DNN(model_features)
    
    checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
    costfunction_errors, earlystop_errors = train_earlystop(
        training_data=training_data,
        training_keys=training_keys_binarized,
        testing_data=testing_data,
        testing_keys=testing_keys_binarized,
        model=model,
        optimizer=optimizer,
        num_epochs=25,
        obj_cost=model.cross_entropy,
        earlystop_cost_fn=model.f1_error,
        earlystop_patience=10,
        verbose=True,
        fit_batch_verbose=1,
        data_augmentation=model.poisson_data_augmentation)

    checkpoint_directory = "final-models"
    model.save_weights('./final-models/DNN_checkpoint_'+str(network_id))

Epoch 1: CostFunc loss: 5.14 5.11, EarlyStop loss: 0.77 0.76
Epoch 2: CostFunc loss: 3.76 3.71, EarlyStop loss: 0.57 0.55
Epoch 3: CostFunc loss: 3.03 2.97, EarlyStop loss: 0.41 0.37
Epoch 4: CostFunc loss: 2.55 2.48, EarlyStop loss: 0.35 0.30
Epoch 5: CostFunc loss: 2.30 2.22, EarlyStop loss: 0.39 0.35
Epoch 6: CostFunc loss: 1.96 1.88, EarlyStop loss: 0.26 0.21
Epoch 7: CostFunc loss: 1.78 1.70, EarlyStop loss: 0.22 0.17
Epoch 8: CostFunc loss: 1.57 1.49, EarlyStop loss: 0.17 0.13
Epoch 9: CostFunc loss: 1.49 1.41, EarlyStop loss: 0.18 0.14
Epoch 10: CostFunc loss: 1.38 1.30, EarlyStop loss: 0.17 0.13
Epoch 11: CostFunc loss: 1.34 1.26, EarlyStop loss: 0.19 0.15
Epoch 12: CostFunc loss: 1.24 1.16, EarlyStop loss: 0.14 0.10
Epoch 13: CostFunc loss: 1.21 1.14, EarlyStop loss: 0.15 0.11
Epoch 14: CostFunc loss: 1.16 1.09, EarlyStop loss: 0.17 0.13
Epoch 15: CostFunc loss: 1.08 1.01, EarlyStop loss: 0.12 0.08
Epoch 16: CostFunc loss: 1.04 0.96, EarlyStop loss: 0.11 0.08
Epoch 17: CostFun